In [1]:
import math, itertools
import pandas as pd
import numpy as np

In [2]:
# Initialize data
measurements = {
    "id": [2, 11, 13],
    "lat_WGS84_dd": [37.97527011, 37.97521406, 37.97519902],
    "lon_WGS84_dd": [23.78003145, 23.78002484, 23.77994047],
    "h_WGS84": [243.00, 238.20, 244.00]
}

constant = {
    "id": [2, 11, 13],
    "lat_GGRS87_dms": ["37,58,21.617", "37,58,21.3995", "37,58,21.2610"],
    "lon_GGRS87_dms": ["23,46,42.0457", "23,46,42.0728", "23,46,41.6921"],
    "h_ortho": [207.098, 206.988, 207.080]
}

_n = 7

# From EGSA87 -> WGS84 transformation parameters
_dx = -199.87
_dy = 74.79
_dz = 246.62

In [3]:
# Create constant points dataframe
const_points = pd.DataFrame(constant)
const_points

,id,lat_GGRS87_dms,lon_GGRS87_dms,h_ortho
0,2,"37,58,21.617","23,46,42.0457",207.098
1,11,"37,58,21.3995","23,46,42.0728",206.988
2,13,"37,58,21.2610","23,46,41.6921",207.080


In [4]:
def dms2dd(d, m, s):
    if d < 360 and m < 60 and s < 60:
        _dd = round(d + m / 60 + s / 3600, 12)
    else:
        _dd = 0
    return(_dd)

def dms2dd_df(df, field):
    angle = df[field].split(",")
    return(dms2dd(float(angle[0]), float(angle[1]), float(angle[2])))

In [5]:
# Convert GGRS87 coords from dms to dd
const_points['lat_GGRS87_dd'] = const_points.apply(dms2dd_df, axis= 1, args= ['lat_GGRS87_dms'])
const_points['lon_GGRS87_dd'] = const_points.apply(dms2dd_df, axis= 1, args= ['lon_GGRS87_dms'])
const_points

,id,lat_GGRS87_dms,lon_GGRS87_dms,h_ortho,lat_GGRS87_dd,lon_GGRS87_dd
0,2,"37,58,21.617","23,46,42.0457",207.098,37.972671,23.778346
1,11,"37,58,21.3995","23,46,42.0728",206.988,37.972611,23.778354
2,13,"37,58,21.2610","23,46,41.6921",207.080,37.972572,23.778248


In [6]:
# Calculate geometric height (h = H + N)
const_points['h_GGRS87'] = const_points['h_ortho'] + _n
const_points

,id,lat_GGRS87_dms,lon_GGRS87_dms,h_ortho,lat_GGRS87_dd,lon_GGRS87_dd,h_GGRS87
0,2,"37,58,21.617","23,46,42.0457",207.098,37.972671,23.778346,214.098
1,11,"37,58,21.3995","23,46,42.0728",206.988,37.972611,23.778354,213.988
2,13,"37,58,21.2610","23,46,41.6921",207.080,37.972572,23.778248,214.080


In [7]:
def lat_lon_to_XYZ(lat, lon, h):
    # GRS80 ellipsoid
    a = 6378137
    f = 0.00335281068
    e = math.sqrt(f * (2 - f))

    lat_rad = lat * math.pi / 180
    lon_rad = lon * math.pi / 180

    _nr = a / (math.sqrt(1 - e**2 * math.sin(lat_rad) * math.sin(lat_rad)))
    _x = (_nr + h) * math.cos(lat_rad) * math.cos(lon_rad)
    _y = (_nr + h) * math.cos(lat_rad) * math.sin(lon_rad)
    _z = ((1 - e**2) * _nr + h) * math.sin(lat_rad)
    return([round(_x, 3), round(_y, 3), round(_z, 3)])

def lat_lon_to_XYZ_df(df, field_lat, field_lon, field_h):
    return(lat_lon_to_XYZ(df[field_lat], df[field_lon], df[field_h]))

In [8]:
# Convert GGRS87 dd coords to XYZ coords
const_points[["X_GGRS87", "Y_GGRS87", "Z_GGRS87"]] = const_points.apply(lat_lon_to_XYZ_df, axis= 1, 
                                                        result_type= "expand",
                                                        args= ["lat_GGRS87_dd", "lon_GGRS87_dd", "h_GGRS87"])
const_points

,id,lat_GGRS87_dms,lon_GGRS87_dms,h_ortho,lat_GGRS87_dd,lon_GGRS87_dd,h_GGRS87,X_GGRS87,Y_GGRS87,Z_GGRS87
0,2,"37,58,21.617","23,46,42.0457",207.098,37.972671,23.778346,214.098,4607099.969,2029893.663,3903184.924
1,11,"37,58,21.3995","23,46,42.0728",206.988,37.972611,23.778354,213.988,4607103.399,2029895.897,3903179.570
2,13,"37,58,21.2610","23,46,41.6921",207.080,37.972572,23.778248,214.080,4607109.616,2029888.482,3903176.260


In [9]:
# Transform GGRS87 XYZ coords to WGS84 XYZ coords
const_points['X_WGS84'] = const_points['X_GGRS87'] + _dx
const_points['Y_WGS84'] = const_points['Y_GGRS87'] + _dy
const_points['Z_WGS84'] = const_points['Z_GGRS87'] + _dz
const_points

,id,lat_GGRS87_dms,lon_GGRS87_dms,h_ortho,lat_GGRS87_dd,lon_GGRS87_dd,h_GGRS87,X_GGRS87,Y_GGRS87,Z_GGRS87,X_WGS84,Y_WGS84,Z_WGS84
0,2,"37,58,21.617","23,46,42.0457",207.098,37.972671,23.778346,214.098,4607099.969,2029893.663,3903184.924,4606900.099,2029968.453,3903431.544
1,11,"37,58,21.3995","23,46,42.0728",206.988,37.972611,23.778354,213.988,4607103.399,2029895.897,3903179.570,4606903.529,2029970.687,3903426.190
2,13,"37,58,21.2610","23,46,41.6921",207.080,37.972572,23.778248,214.080,4607109.616,2029888.482,3903176.260,4606909.746,2029963.272,3903422.880


In [10]:
# Create measure points dataframe
meas_points = pd.DataFrame(measurements)

In [11]:
# Convert WGS84 dd coords to XYZ coords
meas_points[["X_WGS84", "Y_WGS84", "Z_WGS84"]] = meas_points.apply(lat_lon_to_XYZ_df, axis= 1, 
                                                    result_type= "expand",
                                                    args= ["lat_WGS84_dd", "lon_WGS84_dd", "h_WGS84"])
meas_points

,id,lat_WGS84_dd,lon_WGS84_dd,h_WGS84,X_WGS84,Y_WGS84,Z_WGS84
0,2,37.975270,23.780031,243.0,4606898.684,2029966.803,3903430.096
1,11,37.975214,23.780025,238.2,4606898.959,2029966.290,3903422.239
2,13,37.975199,23.779940,244.0,4606907.072,2029961.764,3903424.491


In [12]:
# Calculate WGS84 ΔX, ΔY, ΔΖ between measure - constant points
diff_WGS84 = pd.DataFrame()
diff_WGS84['id'] = meas_points['id'].copy()
diff_WGS84['dx'] = round(meas_points['X_WGS84'] - const_points['X_WGS84'], 4)
diff_WGS84['dy'] = round(meas_points['Y_WGS84'] - const_points['Y_WGS84'], 4)
diff_WGS84['dz'] = round(meas_points['Z_WGS84'] - const_points['Z_WGS84'], 4)
diff_WGS84

,id,dx,dy,dz
0,2,-1.415,-1.650,-1.448
1,11,-4.570,-4.397,-3.951
2,13,-2.674,-1.508,1.611


In [13]:
def geocentric_topocentric(lat, lon, dx, dy, dz):
    lat_rad = lat * math.pi / 180
    lon_rad = lon * math.pi / 180

    dxdydz = np.array([[dx], 
                       [dy], 
                       [dz]])
    dxdydz = np.matrix(dxdydz)
    
    rotation_table = np.array(
                [[-np.sin(lat_rad) * np.cos(lon_rad), -np.sin(lat_rad) * np.sin(lon_rad), np.cos(lat_rad)],
                 [-np.sin(lon_rad), np.cos(lon_rad), 0],
                 [np.cos(lat_rad) * np.cos(lon_rad), np.cos(lat_rad) * np.sin(lon_rad), np.sin(lat_rad)]]
                )
    
    rotation_table = np.matrix(rotation_table)
    
    dndedu = np.round(rotation_table * dxdydz, 3).flatten().tolist()
    return(dndedu)

In [14]:
# Transform WGS84 ΔX, ΔY, ΔΖ to topocentric reference frame
# using each point as origin
diff_topo = pd.DataFrame()
diff_topo['id'] = ["N", "E", "U"]

for i in range(len(meas_points['id'])):
    diff_topo[f"p_{meas_points['id'].iloc[i]}"] = \
        geocentric_topocentric(meas_points['lat_WGS84_dd'].iloc[i], meas_points['lon_WGS84_dd'].iloc[i],
                                 diff_WGS84['dx'].iloc[i], diff_WGS84['dy'].iloc[i], diff_WGS84['dz'].iloc[i])

diff_topo

,id,p_2,p_11,p_13
0,N,0.065,0.550,3.150
1,E,-0.939,-2.181,-0.302
2,U,-2.436,-7.125,-1.417


In [15]:
# Find available point combinations (in order to be used for more than 3 points)
min_id = meas_points['id'].min()
max_id = meas_points['id'].max()
combinations = list(map(list, list(itertools.combinations(meas_points['id'], 2))))
for comb in combinations:
    if comb[0] == min_id and comb[1] == max_id:
        comb.sort(reverse= True)
combinations.sort()
combinations

[[2, 11], [11, 13], [13, 2]]

In [16]:
# Geocentric vectors of measure points
meas_vectors = pd.DataFrame(["DX", "DY", "DZ"], columns=["id"])
for comb in combinations:
    ind0 = meas_points.loc[meas_points['id'] == comb[0]].index[0]
    ind1 = meas_points.loc[meas_points['id'] == comb[1]].index[0]

    meas_vectors[f"v_{comb[0]}_{comb[1]}"] = \
                        [round(meas_points['X_WGS84'].iloc[ind1] - meas_points['X_WGS84'].iloc[ind0], 4),
                         round(meas_points['Y_WGS84'].iloc[ind1] - meas_points['Y_WGS84'].iloc[ind0], 4),
                         round(meas_points['Z_WGS84'].iloc[ind1] - meas_points['Z_WGS84'].iloc[ind0], 4)]
meas_vectors

,id,v_2_11,v_11_13,v_13_2
0,DX,0.275,8.113,-8.388
1,DY,-0.513,-4.526,5.039
2,DZ,-7.857,2.252,5.605


In [17]:
# Set origin point with id = 2
ind0 = meas_points.loc[meas_points['id'] == 2].index[0]
lat = meas_points['lat_WGS84_dd'].iloc[ind0]
lon = meas_points['lon_WGS84_dd'].iloc[ind0]

In [18]:
# Calculate topocentric vectors from origin point 2
base_vectors = pd.DataFrame([], columns= list(meas_vectors.columns))
base_vectors['id'] = ["N", "E", "U"] #, "geo_dist", "topo_dist"]

for index, comb in enumerate(combinations):
    field = f"v_{comb[0]}_{comb[1]}"

    base_vectors[field] = geocentric_topocentric(lat, lon, 
                            meas_vectors[field].iloc[0], meas_vectors[field].iloc[1], meas_vectors[field].iloc[2])
base_vectors

,id,v_2_11,v_11_13,v_13_2
0,N,-6.221,-1.670,7.891
1,E,-0.580,-7.413,7.993
2,U,-4.799,5.799,-1.000


In [19]:
def distance(dx, dy, dz):
    return(round(math.sqrt(dx**2 + dy**2 + dz**2), 3))

In [20]:
# Calculate geocentric & topocentric base distances of measure points
geo_dist = {"id": "geo_dist"}
topo_dist = {"id": "topo_dist"}
for index, comb in enumerate(combinations):
    field = f"v_{comb[0]}_{comb[1]}"

    geo_dist[field] = distance(meas_vectors[field].iloc[0], meas_vectors[field].iloc[1], meas_vectors[field].iloc[2])
    topo_dist[field] = distance(base_vectors[field].iloc[0], base_vectors[field].iloc[1], base_vectors[field].iloc[2])

base_vectors = base_vectors.append(geo_dist, ignore_index= True)
base_vectors = base_vectors.append(topo_dist, ignore_index= True)
base_vectors

,id,v_2_11,v_11_13,v_13_2
0,N,-6.221,-1.670,7.891
1,E,-0.580,-7.413,7.993
2,U,-4.799,5.799,-1.000
3,geo_dist,7.879,9.559,11.277
4,topo_dist,7.878,9.559,11.276


In [21]:
# Geocentric vectors of constant points
const_vectors = pd.DataFrame(["DX", "DY", "DZ"], columns=["id"])
for comb in combinations:
    ind0 = const_points.loc[const_points['id'] == comb[0]].index[0]
    ind1 = const_points.loc[const_points['id'] == comb[1]].index[0]

    const_vectors[f"v_{comb[0]}_{comb[1]}"] = \
                        [round(const_points['X_WGS84'].iloc[ind1] - const_points['X_WGS84'].iloc[ind0], 4),
                         round(const_points['Y_WGS84'].iloc[ind1] - const_points['Y_WGS84'].iloc[ind0], 4),
                         round(const_points['Z_WGS84'].iloc[ind1] - const_points['Z_WGS84'].iloc[ind0], 4)]
const_vectors

,id,v_2_11,v_11_13,v_13_2
0,DX,3.430,6.217,-9.647
1,DY,2.234,-7.415,5.181
2,DZ,-5.354,-3.310,8.664


In [22]:
# Calculate geocentric base distances of constant points
# and the difference between measure distance - constant distance
geo_dist_ = {"id": "geo_dist_"}
geo_dist_diff = {"id": "geo_dist_diff"}
for index, comb in enumerate(combinations):
    ind0 = base_vectors.loc[base_vectors['id'] == "geo_dist"].index[0]
    field = f"v_{comb[0]}_{comb[1]}"

    geo_dist_[field] = distance(const_vectors[field].iloc[0], 
                                const_vectors[field].iloc[1], const_vectors[field].iloc[2])
    geo_dist_diff[field] = base_vectors[field].iloc[ind0] - geo_dist_[field]

const_vectors = const_vectors.append(geo_dist_, ignore_index= True)
const_vectors = const_vectors.append(geo_dist, ignore_index= True)
const_vectors = const_vectors.append(geo_dist_diff, ignore_index= True)
const_vectors

,id,v_2_11,v_11_13,v_13_2
0,DX,3.430,6.217,-9.647
1,DY,2.234,-7.415,5.181
2,DZ,-5.354,-3.310,8.664
3,geo_dist_,6.740,10.227,13.963
4,geo_dist,7.879,9.559,11.277
5,geo_dist_diff,1.139,-0.668,-2.686
